In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2 import *
from dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [2]:
#initialize parameters for 2 layers
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(1)
    
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros(shape=(n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros(shape=(n_y, 1))
    ### END CODE HERE ###
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters    

In [3]:
parameters = initialize_parameters(2,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))


W1 = [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.00865408 -0.02301539]]
b2 = [[0.]]


In [4]:
#initialize parameters deep
def initialize_parameters_deep(layer_dim):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dim)            # number of layers in the network

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.random.randn(layer_dim[l], layer_dim[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dim[l], 1))
        ### END CODE HERE ###
        
        assert(parameters['W' + str(l)].shape == (layer_dim[l], layer_dim[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dim[l], 1))

        
    return parameters
                            

In [5]:
parameters = initialize_parameters_deep([5, 4, 3])
print("W1 = " +str(parameters["W1"]))
print("b1 = " +str(parameters["b1"]))
print("W2 = " +str(parameters["W2"]))
print("b2 = " +str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


In [6]:
# Z^[l] = W^[l] A^[l-1] + b^[l]  --> (1)


In [7]:
#linear forward
def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    ### END CODE HERE ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [8]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[3.1980455  7.85763489]]


In [9]:
# Sigmoid: σ(Z) = σ(WA +b) = 1/1+e^-(WA + b)
 
# This function returns two items: the activation value "a" and a "cache" that contains "Z" (it's what we will feed in to the 
#corresponding backward function). To use it you could just call:
# A, activation_cache = sigmoid(Z)

# ReLU: The mathematical formula for ReLu is A = RELUE(Z) = max(0,Z)
# This function returns two items: the activation value "A" and a "cache" that contains "Z" (it's what we will feed in to the
#corresponding backward function). To use it you could just call:
# A, activation_cache = relu(Z)

# For more convenience, you are going to group two functions (Linear and Activation) into one function (LINEAR->ACTIVATION). 
#Hence, you will implement a function that does the LINEAR forward step followed by an ACTIVATION forward step.

In [10]:
def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)

    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [11]:
A_prev, W, b = linear_activation_forward_test_case()
A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " +str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With relu: A = " +str(A))

With sigmoid: A = [[0.96076066 0.99961336]]
With relu: A = [[3.1980455  7.85763489]]


In [12]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        A, cache = linear_activation_forward(A_prev, 
                                             parameters['W' + str(l)], 
                                             parameters['b' + str(l)], 
                                             activation='relu')
        caches.append(cache)
        
        ### END CODE HERE ###
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL, cache = linear_activation_forward(A, 
                                          parameters['W' + str(L)], 
                                          parameters['b' + str(L)], 
                                          activation='sigmoid')
    caches.append(cache)
    
    ### END CODE HERE ###
    
    assert(AL.shape == (1, X.shape[1]))
            
    return AL, caches

In [13]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)
print("AL = " +str(AL))
print("Length of cache list = " +str(len(caches)))

AL = [[0.0844367  0.92356858]]
Length of cache list = 2


In [14]:
# Cost Function
#-1/m Σ(y^(i) log(a^[L](i)) + (1-y^(i)) log(1-a^[L](i))) --> (2)

In [15]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = (-1/m) * np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1-Y, np.log(1-AL)))
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    return cost
    

In [16]:
Y, AL = compute_cost_test_case()
print("cost =" +str(compute_cost(AL, Y)))

cost =0.41493159961539694


In [17]:
# dZ^[l] = ∂L/∂Z^[l]

# dW^[l] = ∂L/∂W^[l] = 1/m dZ^[l] A^[l-1].T -->eq (3)
# db^[l] = ∂L/∂b^[l] = 1/m Σ dZ^[l](i)      -->eq (4)
# dA^[l-1] = ∂L/∂A^[l-1] = W^[l].T dZ^[l]   -->eq (5)

In [18]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    dW = np.dot(dZ, cache[0].T) / m
    db = np.squeeze(np.sum(dZ, axis=1, keepdims=True)) / m
    dA_prev = np.dot(cache[1].T, dZ)
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (isinstance(db, float))
    
    return dA_prev, dW, db

In [19]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 2.38272385  5.85438014]
 [ 6.31969219 15.52755701]
 [-3.97876302 -9.77586689]]
dW = [[ 2.77870358 -0.05500058 -5.13144969]]
db = 5.527840195


In [20]:
# To help you implement linear_activation_backward, we provided two backward functions:

# sigmoid_backward: Implements the backward propagation for SIGMOID unit. You can call it as follows:
# dZ = sigmoid_backward(dA, activation_cache)

# relu_backward: Implements the backward propagation for RELU unit. You can call it as follows:
# dZ = relu_backward(dA, activation_cache)
# If g(.) is the activation function, sigmoid_backward and relu_backward compute

# dZ^[l] = dA^[l] * g'(Z^[l]) -->(6)

In [21]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache

    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)

    dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

In [22]:
AL, linear_activation_cache = linear_activation_backward_test_case()
dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print("sigmoid:")
print("dA_prev = " +str(dA_prev))
print("dW = " +str(dW))
print("db = " +str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print("relu:")
print("dA_prev = " +str(dA_prev))
print("dW = " +str(dW))
print("db = " +str(db) + "\n")

sigmoid:
dA_prev = [[ 0.08982777  0.00226265]
 [ 0.23824996  0.00600122]
 [-0.14999783 -0.00377826]]
dW = [[-0.06001514 -0.09687383 -0.10598695]]
db = 0.06180098427299098

relu:
dA_prev = [[ 2.38272385  5.85438014]
 [ 6.31969219 15.52755701]
 [-3.97876302 -9.77586689]]
dW = [[ 2.77870358 -0.05500058 -5.13144969]]
db = 5.527840195



In [23]:
# Your code thus needs to compute dAL = ∂L/∂A^[L]
 
# To do so, use this formula (derived using calculus which you don't need in-depth knowledge of):

# dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivative of cost with respect to AL


In [24]:
def L_model_backward(AL, Y, caches):  # AL -- probability vector, output of the forward propagation (L_model_forward())
                                      # Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    ### END CODE HERE ###
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    current_cache = caches[-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_backward(sigmoid_backward(dAL, 
                                                                                                        current_cache[1]), 
                                                                                       current_cache[0])
    ### END CODE HERE ###
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 2)], caches". Outputs: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_backward(sigmoid_backward(dAL, current_cache[1]), current_cache[0])
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        ### END CODE HERE ###

    return grads

In [25]:
X_assess, Y_assess, AL, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

dW1 = [[-0.09686122 -0.04840482 -0.11864308]]
db1 = -0.26259499837854655
dA1 = [[-0.71011462 -0.22925516]
 [-0.17330152 -0.05594909]
 [-0.03831107 -0.01236844]]


In [26]:
# W^[l] = W^[l] - α dW^[l]  --> eq (8)
# b^[l] = b^[l] - α db^[l]  --> eq (9)

In [27]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters)//2
    for l in range(L):
        parameters["W" +str(l+1)] = parameters["W" +str(l+1)] - learning_rate * grads["dW" +str(l+1)]
        parameters["b" +str(l+1)] = parameters["b" +str(l+1)] - learning_rate * grads["db" +str(l+1)]
    return parameters

In [28]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)
print ("W1 = " + str(parameters["W1"]))
print ("b1 = " + str(parameters["b1"]))
print ("W2 = " + str(parameters["W2"]))
print ("b2 = " + str(parameters["b2"]))
print ("W3 = " + str(parameters["W3"]))
print ("b3 = " + str(parameters["b3"]))

W1 = [[ 1.72555789  0.3700272   0.07818896]
 [-1.8634927  -0.2773882  -0.35475898]
 [-0.08274148 -0.62700068 -0.04381817]
 [-0.47721803 -1.31386475  0.88462238]]
b1 = [[-0.07593768]
 [-0.07593768]
 [-0.07593768]
 [-0.07593768]]
W2 = [[ 0.71838378  1.70957306  0.05003364 -0.40467741]
 [-0.54535995 -1.54647732  0.98236743 -1.10106763]
 [-1.18504653 -0.2056499   1.48614836  0.23671627]]
b2 = [[-0.08616376]
 [-0.08616376]
 [-0.08616376]]
W3 = [[-0.88352436 -0.7129932   0.62524497]
 [-0.02025258 -0.76883635 -0.23003072]]
b3 = [[0.08416196]
 [0.08416196]]
